In [2]:
import pandas as pd
import datetime
import unfolded_query_helper

# Get you access key at https://covid.rilldata.com/accesstoken

Note: you have to get your gmail account approved to access the SG data from this database, to get approved message Mike Driscoll (Rill Data) on Slack

In [3]:
token = "ENTER YOUR TOKEN"

All query functions return a pandas dataframe

Initialize an instance of the class in order to get started  

In [4]:
db = unfolded_query_helper.session(token)

## Query by date range

In [6]:
print(db.query_time("2020-04-19","2020-04-26",state="ohio",limit=5))

Status Code: 200  (200 is what you want)
     County_State  GEOID                    __time  age_50_to_69  \
0    Clermont, OH  39025 2020-04-19 00:00:00+00:00             0   
1        Lake, OH  39085 2020-04-19 00:00:00+00:00             0   
2       Lucas, OH  39095 2020-04-19 00:00:00+00:00             0   
3        Erie, OH  39043 2020-04-19 00:00:00+00:00             0   
4  Montgomery, OH  39113 2020-04-19 00:00:00+00:00             0   

   age_70_and_over  age_under_50  at_home_hour00  at_home_hour01  \
0                0             0               0               0   
1                0             0               0               0   
2                0             0               0               0   
3                0             0               0               0   
4                0             0               0               0   

   at_home_hour02  at_home_hour03  ...  \
0               0               0  ...   
1               0               0  ...   
2              


**Query by date range and county, state** (or if you want to get a given county from all states use: county = 'countyName%')


In [7]:
print(db.query_time("2020-04-19","2020-04-26",county='Lucas, OH').head(5))

Status Code: 200  (200 is what you want)
  County_State  GEOID                    __time  age_50_to_69  \
0    Lucas, OH  39095 2020-04-19 00:00:00+00:00             0   
1    Lucas, OH  39095 2020-04-19 00:00:00+00:00            33   
2    Lucas, OH  39095 2020-04-19 00:00:00+00:00            39   
3    Lucas, OH  39095 2020-04-19 00:00:00+00:00            49   
4    Lucas, OH  39095 2020-04-19 00:00:00+00:00            54   

   age_70_and_over  age_under_50  at_home_hour00  at_home_hour01  \
0                0             0               0               0   
1               31           251               4               3   
2               15           177              13              13   
3                8           257               4               4   
4               19           149               5               3   

   at_home_hour02  at_home_hour03  ...  \
0               0               0  ...   
1               4               1  ...   
2              13              13

**Query by range, but only return selected variables**

Note: the temperature data is pretty inconsistent and is in Kelvin

In [8]:
print(db.query_time("2020-04-19","2020-04-26", 
                    variables= "County_State, __time, cases, device_count, max_temperature",
                    county='Lucas, OH').head(5))

Status Code: 200  (200 is what you want)
  County_State                    __time     cases  device_count  \
0    Lucas, OH 2020-04-19 00:00:00+00:00  0.000000             0   
1    Lucas, OH 2020-04-19 00:00:00+00:00  0.573449             7   
2    Lucas, OH 2020-04-19 00:00:00+00:00  0.420529            29   
3    Lucas, OH 2020-04-19 00:00:00+00:00  0.571628            13   
4    Lucas, OH 2020-04-19 00:00:00+00:00  0.404145            14   

   max_temperature  
0              0.0  
1            280.8  
2            280.8  
3            280.8  
4            280.8  


**Query by only county and state** (no time range)

In [13]:
print(db.query_county_state('Los Angeles','CA',variables= "County_State, __time, cases",limit=5))
# or use full state name and let the code change it to a 2 letter abbreviation for you
print(db.query_county_state('Los Angeles','California',variables= "County_State, __time, cases",limit=5))

Status Code: 200  (200 is what you want)
      County_State                    __time  cases
0  Los Angeles, CA 2020-01-01 00:00:00+00:00      0
1  Los Angeles, CA 2020-01-01 00:00:00+00:00      0
2  Los Angeles, CA 2020-01-01 00:00:00+00:00      0
3  Los Angeles, CA 2020-01-01 00:00:00+00:00      0
4  Los Angeles, CA 2020-01-01 00:00:00+00:00      0
Status Code: 200  (200 is what you want)
      County_State                    __time  cases
0  Los Angeles, CA 2020-01-01 00:00:00+00:00      0
1  Los Angeles, CA 2020-01-01 00:00:00+00:00      0
2  Los Angeles, CA 2020-01-01 00:00:00+00:00      0
3  Los Angeles, CA 2020-01-01 00:00:00+00:00      0
4  Los Angeles, CA 2020-01-01 00:00:00+00:00      0


## Query by any condition 

This is just building the WHERE statement in an SQL query

You can either build the WHERE statement manually or use the where_builder to help build it (only helps you avoid triple nested quotes)

In [14]:
print(db.query_any_condition('''"total_pop" > 10000 AND "cases" > 100''',
                             variables= "County_State, __time, cases, device_count, max_temperature",
                             limit = 5))

Status Code: 200  (200 is what you want)
  County_State                    __time       cases  device_count  \
0     Cass, ND 2020-05-03 00:00:00+00:00  128.655580          2578   
1     Cass, ND 2020-04-28 00:00:00+00:00  100.372620          2715   
2     Cass, ND 2020-04-29 00:00:00+00:00  104.625694          2654   
3     Cass, ND 2020-05-02 00:00:00+00:00  121.638000          2756   
4     Cass, ND 2020-04-30 00:00:00+00:00  109.091420          2592   

   max_temperature  
0            286.1  
1            290.1  
2            288.0  
3            294.6  
4            291.2  


In [17]:
# using where builder
print('where builder return:',db.where_builder(['"total_pop" > 10000','"cases" > 100']))
print()
print(db.query_any_condition(db.where_builder(['"total_pop" > 10000','"cases" > 100']),
                             variables= "County_State, __time, cases, device_count, max_temperature",
                             limit = 5))

where builder return: "total_pop" > 10000 AND "cases" > 100

Status Code: 200  (200 is what you want)
  County_State                    __time       cases  device_count  \
0     Cass, ND 2020-05-03 00:00:00+00:00  128.655580          2578   
1     Cass, ND 2020-04-28 00:00:00+00:00  100.372620          2715   
2     Cass, ND 2020-04-29 00:00:00+00:00  104.625694          2654   
3     Cass, ND 2020-05-02 00:00:00+00:00  121.638000          2756   
4     Cass, ND 2020-04-30 00:00:00+00:00  109.091420          2592   

   max_temperature  
0            286.1  
1            290.1  
2            288.0  
3            294.6  
4            291.2  


## Build an advanced query manually

There is also support to build an advanced query manually, but it requires more knowledge of SQL and if you have that then it will most likely easier to just build the entire query statement yourself

**truncate_time** 

Really helpful function if you want to easily remove the large number of zeros in '__time'

In [18]:
sql = db.build_sql_query(["County_State, __time, cases, deaths",
                    '''__time >= TIMESTAMP '2020-06-10' AND __time < TIMESTAMP '2020-06-17' AND County_State = 'Lucas, OH' ''',
                    "ALL"])
df = db.query_default(sql)
print(df.head(5))
print()
# truncate time to not be so ugly (keeps only the date, removes time)
df = db.truncate_time(df)
print(df.head(5))

Status Code: 200  (200 is what you want)
  County_State                    __time      cases    deaths
0    Lucas, OH 2020-06-10 00:00:00+00:00   0.000000  0.000000
1    Lucas, OH 2020-06-10 00:00:00+00:00  18.743736  2.117493
2    Lucas, OH 2020-06-10 00:00:00+00:00   1.896983  0.214304
3    Lucas, OH 2020-06-10 00:00:00+00:00   1.737063  0.196237
4    Lucas, OH 2020-06-10 00:00:00+00:00   1.273846  0.143907

  County_State      __time      cases    deaths
0    Lucas, OH  2020-06-10   0.000000  0.000000
1    Lucas, OH  2020-06-10  18.743736  2.117493
2    Lucas, OH  2020-06-10   1.896983  0.214304
3    Lucas, OH  2020-06-10   1.737063  0.196237
4    Lucas, OH  2020-06-10   1.273846  0.143907


## Using SQL functions in a query 

The below code is only an example of how to use a function in an SQL query

In [19]:
# SQL query to average the cases and deaths on a given date range
sql2 = db.build_sql_query(["""County_State, __time, AVG(cases) as "AVG cases", AVG(deaths) as "AVG deaths" """,
                    '''__time >= TIMESTAMP '2020-06-10' AND __time < TIMESTAMP '2020-06-17' AND County_State = 'Lucas, OH' 
                    GROUP BY 1,2''', # GROUP BY gets introduced because we are using functions in selection
                    "ALL"])
df2 = db.query_default(sql2)
# truncate time
df2 = db.truncate_time(df2)
print(df2.head(10))

Status Code: 200  (200 is what you want)
  County_State      __time  AVG cases  AVG deaths
0    Lucas, OH  2020-06-10   6.005025    0.678392
1    Lucas, OH  2020-06-11   6.015075    0.683417
2    Lucas, OH  2020-06-12   6.057789    0.683417
3    Lucas, OH  2020-06-13   6.077889    0.703518
4    Lucas, OH  2020-06-14   6.100503    0.703518
5    Lucas, OH  2020-06-15   6.173367    0.708543
6    Lucas, OH  2020-06-16   6.180904    0.718593


## Using SQL concatenate to combine columns

In this example we combine the latitude and longitude into 1 column called 'lat, long' (we call it that by referring to it with 'as' in the query)

In [20]:
# SQL query to get the more regional data 
sql3 = db.build_sql_query(["""County_State, __time, cases, deaths, total_pop,origin_census_block_group,CONCAT(lat,', ',lng) as "lat, long" """,
                    '''__time >= TIMESTAMP '2020-06-10' AND __time < TIMESTAMP '2020-06-17' AND County_State = 'Lucas, OH' ''',
                    "ALL"])
df3 = db.query_default(sql3)
df3 = db.truncate_time(df3)
print(df3.head(15))

Status Code: 200  (200 is what you want)
   County_State      __time      cases    deaths  total_pop  \
0     Lucas, OH  2020-06-10   0.000000  0.000000          0   
1     Lucas, OH  2020-06-10  18.743736  2.117493       3399   
2     Lucas, OH  2020-06-10   1.896983  0.214304        344   
3     Lucas, OH  2020-06-10   1.737063  0.196237        315   
4     Lucas, OH  2020-06-10   1.273846  0.143907        231   
5     Lucas, OH  2020-06-10   2.007273  0.226763        364   
6     Lucas, OH  2020-06-10   4.384016  0.495265        795   
7     Lucas, OH  2020-06-10   1.731548  0.195614        314   
8     Lucas, OH  2020-06-10   1.224216  0.138301        222   
9     Lucas, OH  2020-06-10   4.742457  0.535759        860   
10    Lucas, OH  2020-06-10   1.450310  0.163843        263   
11    Lucas, OH  2020-06-10   2.238881  0.252928        406   
12    Lucas, OH  2020-06-10   3.303177  0.373162        599   
13    Lucas, OH  2020-06-10   2.360200  0.266633        428   
14    Lucas, O



Sorry for some of the really long tables at the start, it's my first creating my own notebook, so I don't know all the formatting tricks.

I hope this code helps some people. It's worth to note that this code was written on Windows 10, so I can't ensure it will operate as intended on other operating system. I tried to keep unfolded_sql_helper well documented (maybe too much), hopefully enough to let you understand what is being done behind the scenes.